In [121]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

from src import modelers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Prediction algorithms (supervised learning)
1. Linear Regression for predicted population count for each winner (multivariate linear regression)


1. Logistic (get beta values, or log odds ratio and look at predict proba)
2. KNN
3. Gradient Boosting Classifier (Random Forest)
Don't forget confusion matrix!!

4. We need state results (and electoral college votes) to see if they match for sum of county results.

Can we get updated demographic information to predict the 2020 elections?

5. Can we scrape twitter data and look at popularity? Maybe use NLP/NMF to look at latent features for the different tweets? Too ambitious....
6. Hate crime statistics? Starter code to download from github is in data_clean script




In [39]:
# Import data
pd.set_option("display.max_columns", 40)
data = pd.read_csv('data/cleaned_data2.csv')
data.head()

,Unnamed: 0,state,county,fips,white_pct,black_pct,hispanic_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,rural_pct,trump16_pct,clinton16_pct,otherpres16_pct,romney12_pct,obama12_pct,otherpres12_pct,cvap_pct12,metro,rural,urban_metroadj,urban_not_metroadj
0,0,Alabama,Autauga,1001,75.683482,18.370906,2.572254,1.838362,51.176225,40.037058,13.978456,53099.0,5.591657,12.417046,75.407229,42.002162,72.766588,23.769671,3.463741,72.618252,26.587832,0.793916,58.815434,1,0,0,0
1,1,Alabama,Baldwin,1003,83.178788,9.225603,4.366698,3.269510,51.194928,35.474412,18.714851,51365.0,6.286843,9.972418,70.452889,42.279099,76.545712,19.385601,4.068687,77.358269,21.589444,1.052286,56.228504,1,0,0,0
2,2,Alabama,Barbour,1005,45.885624,47.888329,4.309762,2.859397,46.498084,37.664387,16.528895,33956.0,12.824738,26.235928,87.132213,67.789635,52.096666,46.527844,1.375490,48.223130,51.368494,0.408376,56.485890,0,0,1,0
3,3,Alabama,Bibb,1007,74.765196,21.212121,2.223994,1.351232,46.464646,37.329435,14.885699,39776.0,7.146827,19.301587,88.000000,68.352607,76.403220,21.249575,2.347205,72.826603,26.152019,1.021378,47.868107,1,0,0,0
4,4,Alabama,Blount,1009,87.657701,1.557951,8.727298,4.271801,50.485235,37.240053,17.192916,46212.0,5.953833,19.968585,86.950243,89.951502,89.334844,8.425825,2.239331,86.465884,12.371907,1.162209,56.577893,1,0,0,0


In [40]:
data.drop(columns=['Unnamed: 0', 'state', 'county', 'fips', 'otherpres12_pct'], inplace=True)

In [41]:
# Split into metro, rural, urban 
metro = data[data['metro'] == 1]
rural = data[data['rural'] == 1]
urban_metroadj = data[data['urban_metroadj'] == 1]
urban_not_metroadj = data[data['urban_not_metroadj'] == 1]

In [42]:
def train_test_split_rep(df):

    # Pop ys
    trump = df.pop('trump16_pct').values()
    clinton = df.pop('clinton16_pct').values()
    other = df.pop('otherpres16_pct').values()
    X = df.values()
        
    # Make train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, trump)
    
    

In [43]:
# Extract our labels
trump = data.pop('trump16_pct')
clinton = data.pop('clinton16_pct')
other = data.pop('otherpres16_pct')

In [162]:
# Make our X and y and do 75%/25% split
X = data
y = trump

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [182]:
# Keep track of indices
train_idx = X_train.index.values
test_idx = X_test.index.values

## Test toy model before using Kfold

In [183]:
# Standardize and fit model
X_trains, X_tests = modelers.standardize(X_train, X_test)
rmse, y_hat, model_coeff = modelers.lin_regression(X_trains, X_tests, y_train, y_test)
rmse, model_coeff

(3.475020783258963,
 array([ 1.88174921, -0.63869564, -0.7334311 , -0.24854546,  0.16289923,
        -0.43870025,  0.29295766,  0.63736309, -0.52632567,  0.44594078,
         3.77261734,  0.46148428,  8.82619963, -2.78940513, -0.67621712,
        -0.18214836,  0.16283229,  0.01463635,  0.04558101]))

In [199]:
y_train.values

array([69.75217115, 47.59511528, 37.99789382, ..., 73.65616675,
       55.28692208, 55.45667447])

In [184]:
# Put into dataframe
coeff = pd.DataFrame(data = model_coeff, index=data.columns, columns=["beta"])

In [185]:
# Get p-values by using stats models 
model = sm.OLS(y_train, X_trains).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            trump16_pct   R-squared (uncentered):                   0.054
Model:                            OLS   Adj. R-squared (uncentered):              0.047
Method:                 Least Squares   F-statistic:                              7.367
Date:                Wed, 13 May 2020   Prob (F-statistic):                    6.93e-19
Time:                        14:57:24   Log-Likelihood:                         -12990.
No. Observations:                2333   AIC:                                  2.602e+04
Df Residuals:                    2315   BIC:                                  2.612e+04
Df Model:                          18                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.8817      4.175      0.451      0.652      -6.305      10.069
x2            -0.6387      3.214     -0.199      0.843      -6.942       5.665
x3            -0.7334      3.291     -0.223      0.824      -7.187       5.720
x4            -0.2485      2.447     -0.102      0.919      -5.047       4.550
x5             0.1629      1.494      0.109      0.913      -2.767       3.092
x6            -0.4387      2.838     -0.155      0.877      -6.003       5.126
x7             0.2930      3.019      0.097      0.923      -5.627       6.213
x8             0.6374      2.505      0.254      0.799      -4.276       5.551
x9            -0.5263      1.844     -0.285      0.775      -4.143       3.091
x10            0.4459      2.664      0.167      0.867      -4.778       5.670
x11            3.7726      2.483      1.519      0.129      -1.097       8.642
x12            0.4615      2.477      0.186      0.852      -4.397       5.320
x13            8.8262     15.530      0.568      0.570     -21.628      39.280
x14           -2.7894     15.863     -0.176      0.860     -33.897      28.318
x15           -0.6762      1.947     -0.347      0.728      -4.493       3.141
x16           -0.1821      1.187     -0.153      0.878      -2.510       2.145
x17            0.1628      1.493      0.109      0.913      -2.766       3.091
x18            0.0146      1.015      0.014      0.988      -1.976       2.005
x19            0.0456      1.134      0.040      0.968      -2.178       2.269
==============================================================================
Omnibus:                     1144.320   Durbin-Watson:                   0.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15192.698
Skew:                          -1.979   Prob(JB):                         0.00
Kurtosis:                      14.858   Cond. No.                     1.14e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.19e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [186]:
# Add to df
coeff['p-values'] = model.pvalues.values
coeff = coeff.apply(lambda x: round(x,2))
coeff.sort_values(by='beta', ascending=False)

,beta,p-values
romney12_pct,8.83,0.57
lesscollege_pct,3.77,0.13
white_pct,1.88,0.65
median_hh_inc,0.64,0.80
rural_pct,0.46,0.85
lesshs_pct,0.45,0.87
age65andolder_pct,0.29,0.92
female_pct,0.16,0.91
rural,0.16,0.91
urban_not_metroadj,0.05,0.97


## KFold with linear regression

In [200]:
fold_scores, coeffs = modelers.kfold_scores(X_trains, y_train, 5)
fold_scores, fold_scores.mean()

(array([3.05800275, 3.85412238, 3.15992682, 3.28940662, 3.00294433]),
 3.2728805805631977)

In [201]:
coeffs.mean(axis=0)

array([ 1.87124985e+00, -6.47458145e-01, -7.42862883e-01, -2.49201483e-01,
        1.63999927e-01, -4.39778136e-01,  2.98089105e-01,  6.39924379e-01,
       -5.30994350e-01,  4.52559053e-01,  3.77188763e+00,  4.61988220e-01,
        8.83569899e+00, -2.77774592e+00, -6.77331991e-01,  1.02427281e+13,
        8.48285009e+12,  9.34939651e+12,  7.62539418e+12])

## Feature Engineering

array([69.56236999, 66.33878861, 74.61092066, 77.5331087 , 48.64838169,
       61.0562471 , 71.30750413, 69.4396434 , 42.09860954, 74.20174924,
       58.49824319, 37.72017595, 76.39530109, 88.11911434, 73.82279982,
       76.89235591, 69.94118112, 59.70302667, 69.7444764 , 67.95231889,
       84.43013091, 66.4459461 , 52.2950305 , 74.93742808, 38.01321768,
       72.62594469, 68.6909341 , 77.83419243, 51.65376247, 47.07192661,
       50.81119633, 72.36484412, 31.39700466, 75.79491547, 78.96047447,
       51.82532531, 73.18422436, 64.52665877, 56.04378104, 59.11886074,
       64.68607676, 78.24998881, 69.24267772, 66.85810584, 45.72754999,
       66.33946118, 59.21823252, 72.35965687, 69.70233029, 79.32531854,
       62.76431244, 70.20245759, 57.19503785, 44.7334393 , 60.22706373,
       69.47558058, 68.77680857, 53.86343873, 46.43162514, 74.27400253,
       80.73008738, 79.21014072, 37.40610135, 91.40641155, 80.34251646,
       76.32348422, 22.41193826, 64.03607078, 24.50945008, 77.28